# 2. Merge data for analysis 
Section 1. 
intersect 





I made an environment on the cluster called soy_data_setup 
programs 
bcftools
vcftools
see conda list -n soy_data_setup
and look in workflow/envs for the soy_data_setup.yaml

Here im going to generate the files needed for the analysis by intersecting not imputing.


data:
soysnp50k_wm82.a2_41317.vcf.gz
and 
cca_sbpa.vcf.gz



In [ ]:
bcftools index soysnp50k_wm82.a2_41317.vcf.gz

bcftools view -S founders.txt soysnp50k_wm82.a2_41317.vcf.gz -Oz -o founders.vcf.gz

bcftools index founders.vcf.gz

I need the chromosomes to be called the same in both files

In [ ]:
#rename chromosomes
bcftools annotate --rename-chrs chr_names.txt founders.vcf.gz -Oz -o founders_gm.vcf.gz

#index
bcftools index founders_gm.vcf.gz

#only include chromosomes
bcftools view founders_gm.vcf.gz --regions Gm01,Gm02,Gm03,Gm04,Gm05,Gm06,Gm07,Gm08,Gm09,Gm10,Gm11,Gm12,Gm13,Gm14,Gm15,Gm16,Gm17,Gm18,Gm19,Gm20 -Oz -o founders_Gm_only.vcf.gz

bcftools index founders_Gm_only.vcf.gz

#check
bcftools query -l founders_Gm_only.vcf.gz
bcftools query -f '%CHROM\n' founders_Gm_only.vcf.gz|uniq

how meny positions SNPs are there?
bcftools query -f '%POS\n' cca_sbpa_Gm_only.vcf.gz | wc -l
9677024
founders_gm.vcf.gz
42195

Do the same with the bigger file with a script.
Try to submit as job: /workflow/scripts/filter_incl_chr.sh
sbatch filter_incl_chr.sh

output is cca_sbpa_Gm_only.vcf.gz


Stats:

I want to get some stats on my files at this point.

file with snps: 

bcftools stats cca_sbpa_Gm_only.vcf.gz > stats_cca.vchk

plot-vcfstats file.vchk -p plots/



Now i will intersect the two files so as to merge them and only retain the 50k SNP sequence. 

In [ ]:
#see isec.sh in scripts directory

bcftools isec cca_sbpa_Gm_only.vcf.gz founders_Gm_only.vcf.gz -p output_dir

#the second file generated called 0002.vcf.gz contains the SNPs that are present in both input files.

bcftools view output_dir/0002.vcf.gz -o intersect_cca_sbpa_f.vcf.gz -Oz

